# Homework 5

Spencer Pease  
ASTR 427

----------------------------

In [1]:
# Imports

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

## Problem 1

## Problem 2